In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import StanfordTagger
from src.data.import_data import import_data
from src.data.clean_data import data_cleansing
from nltk.stem import WordNetLemmatizer
from HanTa import HanoverTagger as ht
from src.features.build_features import feature_generation
from src.data.make_dataset import get_sample

#### Load data into Dataframe

In [2]:
# import data
df=import_data()

In [4]:
df_sample=get_sample(df,300000)

#### select 300.000 entries and denote them with 1 for right translation, then take the next 300.000 entries and shift the lines, so that we have negative examples, afterwards denote them by one

In [5]:
df_sample

,English,German,Translation
0,Resumption of the session,Wiederaufnahme der Sitzungsperiode,1.0
1,I declare resumed the session of the European ...,"Ich erkläre die am Freitag, dem 17. Dezember u...",1.0
2,"Although, as you will have seen, the dreaded '...","Wie Sie feststellen konnten, ist der gefürchte...",1.0
3,You have requested a debate on this subject in...,Im Parlament besteht der Wunsch nach einer Aus...,1.0
4,"In the meantime, I should like to observe a mi...",Heute möchte ich Sie bitten - das ist auch der...,1.0
...,...,...,...
599995,I believe that the broader concept of security...,Verletzungen der Freiheit und Würde des Indivi...,0.0
599996,"Behind structural security risks, there are of...",Kurzum: Die in der Sicherheitsstrategie genann...,0.0
599997,"To sum up, the actions referred to in the Secu...",Eines der Beispiele dafür wäre z. B. Afghanist...,0.0
599998,"One example of this is Afghanistan, and we hop...",Hier hat die Union mit ihrem einzigartigen Ins...,0.0


#### Data Cleansing: Tokenize by white spaces, lowercase, stopword removal

In [6]:
### get first 10.000 entries to test and get features
df_selected = df_sample.sort_index().loc[0:10000]

In [7]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)
df_selected.head(100)

,English,German,Translation
0,Resumption of the session,Wiederaufnahme der Sitzungsperiode,1.0
1,"I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant fest...","Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie...",1.0
2,"Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.","Wie Sie feststellen konnten, ist der gefürchtete ""Millenium-Bug "" nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.",1.0
3,"You have requested a debate on this subject in the course of the next few days, during this part-session.",Im Parlament besteht der Wunsch nach einer Aussprache im Verlauf dieser Sitzungsperiode in den nächsten Tagen.,1.0
4,"In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the vari...","Heute möchte ich Sie bitten - das ist auch der Wunsch einiger Kolleginnen und Kollegen -, allen Opfern der Stürme, insbesondere in den verschiedenen Ländern der Europäischen Union, in einer Schwei...",1.0
...,...,...,...
95,There was a vote on this matter.,Es gab eine Abstimmung zu diesem Punkt.,1.0
96,"As I recall, the outcome of this vote was 422 votes to 180 with a few abstentions.",Diese Abstimmung ist meiner Erinnerung nach so ausgegangen: 422 gegen 180 Stimmen bei einigen wenigen Enthaltungen.,1.0
97,"This means that all the Groups with the exception of the non-attached Members - but, of course, they are not a Group - were in agreement; only your Group thought that we should proceed as you have...","Das heißt, alle Fraktionen, mit Ausnahme der Fraktionslosen - aber die sind ja keine Fraktion - waren sich einig, nur Ihre Fraktion war der Meinung, so zu verfahren, wie Sie es hier vorgeschlagen ...",1.0
98,All of the others were of a different opinion.,Alle anderen waren anderer Meinung.,1.0


In [ ]:
df_sel=data_cleansing(df_selected)

In [ ]:
df_sel

#### Feature Generation

In [ ]:
df_selected=feature_generation(df_sel)

In [ ]:
df_selected

### Add direct translation of German to English and English to German

Die Übersetzung ist ein krasser Zeitaufwand und daher vorerst mit diesen Package nicht feasible.

In [ ]:
# import data
df = import_data()
df_selected = df.sort_index().loc[0:10000]

In [ ]:
import googletrans
from googletrans import Translator
translator = Translator()

In [ ]:
# df_selected["English_to_German"] = df_selected["English"].apply(lambda text: translator.translate(text, src="en", dest='de').text)

In [ ]:
# df_selected["German_to_English"] = df_selected["English"].apply(lambda text: translator.translate(text, src="en", dest='de').text)

### Add segment analysis for german and english

In [ ]:
from textblob import TextBlob
from textblob_de import TextBlobDE

In [ ]:
df_selected["English_Blob"] = df_selected.English.apply(lambda x: TextBlob(x))
df_selected["English_Polarity"] = df_selected.English_Blob.apply(lambda x: x.sentiment.polarity)
df_selected["English_Subjectivity"] = df_selected.English_Blob.apply(lambda x: x.sentiment.subjectivity)

In [ ]:
df_selected["German_Blob"] = df_selected.German.apply(lambda x: TextBlobDE(x))
df_selected["German_Polarity"] = df_selected.German_Blob.apply(lambda x: x.sentiment.polarity)
df_selected["German_Subjectivity"] = df_selected.German_Blob.apply(lambda x: x.sentiment.subjectivity)

In [ ]:
df_selected

### Add Named Entity Recognition

In [ ]:
import spacy
nlp_en = spacy.load("en_core_web_sm")
nlp_de = spacy.load("de_core_news_sm")

In [ ]:
def get_ner_text_en(text):
    doc = nlp_en(text)
    test = []
    for ent in doc.ents:
        test.append([ent.text,  ent.label_])
    return test

In [ ]:
def get_ner_text_de(text):
    doc = nlp_de(text)
    test = []
    for ent in doc.ents:
        test.append([ent.text,  ent.label_])
    return test

In [ ]:
df_selected["English_NER"] = df_selected.English.apply(lambda x: get_ner_text_en(x))
df_selected["German_NER"] = df_selected.German.apply(lambda x: get_ner_text_de(x))

In [ ]:
df_selected